In [103]:
import h5py
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import timm  # timm库有更丰富的预训练模型
from torch import nn, optim
from tqdm import tqdm
import random
# 设置文件路径并得到数据集
train_data_file = "../CNN/data/train_data/"
test_data_file = "../CNN/data/test_data/"
# model_path   torch.save(model.state_dict(), '../CNN/Cmodel_weights.pth')
model_path = "../CNN/"

# 超参数
learning_rate = 1e-6  # 1e-4
weight_decay = 1e-3
batch_size = 16
n_epochs = 50
best_acc= 0
#提取数据
def data_image_label(mode):  #shuffle
        if mode == 'train':
            for i in range(1, 9):
                date_path = f"{train_data_file}" + "traindata" + f"{i}" + ".mat"
                if i != 1:
                    data_image = np.append(data_image, np.array(h5py.File(date_path)["video"], dtype=np.float32),axis=0)
                    data_label = np.append(data_label, np.array(h5py.File(date_path)["label"], dtype=np.float32),axis=0)
                else:
                    data_image = np.array(h5py.File(date_path)["video"], dtype=np.float32)
                    data_label = np.array(h5py.File(date_path)["label"], dtype=np.float32)
            #shuffle
            index = [i for i in range(len(data_image))]
            random.shuffle(index)
            random.shuffle(index)
            data_image = data_image[index,:,:,:]
            data_label = data_label[index,:]


            image_tensor = torch.tensor(data_image, dtype=torch.float32)
            label_tensor = torch.tensor(data_label.squeeze(), dtype=torch.long)
            real_len = len(image_tensor)
            print(image_tensor.shape, label_tensor.shape)
            #  #valid_date
            # valid_image_tensor=image_tensor[:self.valid_len,:,:,:]
            # valid_label_tensor=label_tensor[:self.valid_len]
            # #train_date
            # image_tensor=self.image_tensor[self.valid_len:,:,:,:]
            # label_tensor=self.label_tensor[self.valid_len:]
            # real_len=len(self.image_tensor)
            return image_tensor,label_tensor,real_len
        elif mode == "test":
            for i in range(1, 5):
                date_path = f"{test_data_file}" + "testdata" + f"{i}" + "_label.mat"
                if i != 1:
                    data_image = np.append(data_image, np.array(h5py.File(date_path)["video"], dtype=np.float32),axis=0)
                    data_label = np.append(data_label, np.array(h5py.File(date_path)["label"], dtype=np.float32),axis=0)
                else:
                    data_image = np.array(h5py.File(date_path)["video"], dtype=np.float32)
                    data_label = np.array(h5py.File(date_path)["label"], dtype=np.float32)
            image_tensor = torch.tensor(data_image, dtype=torch.float32)
            label_tensor = torch.tensor(data_label.squeeze(), dtype=torch.long)
            real_len = len(image_tensor)
            print(image_tensor.shape, label_tensor.shape)
            return image_tensor,label_tensor,real_len
train_image_tensor,train_label_tensor,train_real_len=data_image_label('train')
test_image_tensor,test_label_tensor,test_real_len=data_image_label('test')

class SAPI_FMF_Dataset(Dataset):
    def __init__(self,image_data,label_data,data_len,mode="train", valid_ratio=0):
        """
        Args:
            file_path (string): 图像文件 路径
            mode (string): 训练模式还是测试模式
            valid_ratio (float): 验证集比例
        """
        self.image_tensor=image_data
        self.label_tensor=label_data
        self.real_len=data_len
        self.mode = mode
        self.valid_ratio = valid_ratio
        self.valid_len=int(self.valid_ratio*self.real_len)
        if self.mode == 'train':
            self.image_tensor=self.image_tensor[self.valid_len:,:,:,:]
            self.label_tensor=self.label_tensor[self.valid_len:]
            self.real_len=len(self.image_tensor)
        elif self.mode == 'valid':
            self.valid_image_tensor=self.image_tensor[:self.valid_len,:,:,:]
            self.valid_label_tensor=self.label_tensor[:self.valid_len]
            self.real_len=len(self.valid_image_tensor)
        elif self.mode == 'test':
            pass
        print('Finished reading the {} set of Leaves Dataset ({} samples found)'.format(mode, self.real_len))
    def __getitem__(self, index):
        # 从 image_arr中得到索引对应的文件名
        return self.image_tensor[index, :, :, :], self.label_tensor[index]
    def __len__(self):
        return self.real_len  # self.real_len = len(self.image_arr) 返回的是训练/验证/测试/图像的数量
train_dataset = SAPI_FMF_Dataset(train_image_tensor,train_label_tensor,train_real_len,mode='train')
valid_dataset = SAPI_FMF_Dataset( train_image_tensor,train_label_tensor,train_real_len,mode='valid')
test_dataset = SAPI_FMF_Dataset( test_image_tensor,test_label_tensor,test_real_len,mode='test')


# 定义data loader
train_Dataloader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True,drop_last=True)  # 打开乱序  Falsenum_workers=0)
valid_Dataloader = torch.utils.data.DataLoader(dataset=valid_dataset,batch_size=batch_size,shuffle=False,drop_last=True)   #打开乱序  Falsenum_workers=0)
test_Dataloader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False,drop_last=False)
# GPU计算
def get_device():
    return 'cuda' if torch.cuda.is_available() else 'cpu'
device = get_device()
print(device)

model = timm.create_model("resnet10t",num_classes=2,in_chans=3,pretrained=True,output_stride=8,drop_rate=.5,drop_path_rate=0.5,drop_block_rate=0)
# model = timm.create_model("resnet18")
model.fc = nn.Sequential(
    nn.Dropout(.5),
    nn.Linear(model.fc.in_features, 2),
    nn.Dropout(.5)
)
model.to(device)
model.device = device
optimizer = optim.AdamW(model.parameters(), lr=1e-4,
                        weight_decay=weight_decay)  # lr=learning_rate,weight_decay= weight_decay
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=2)
print(model.parameters)
# train
# 对于分类任务，我们使用交叉熵作为性能的度量。
criterion = nn.CrossEntropyLoss()


for epoch in range(n_epochs):
    # 记录训练信息
    model.train()
    train_loss = []
    train_accs = []
    for batch in tqdm(train_Dataloader):
        # ---------- Training ----------  以下是训练模型
        imgs, labels = batch
        # print(imgs.shape)

        logits = model(imgs)  # 将图形数据带入模型计算预测
        # 计算交叉熵损失
        loss = criterion(logits, labels)
        # 清除上一步中存储在参数中的梯度。
        optimizer.zero_grad()
        # 计算参数的梯度。
        loss.backward()
        # 用计算的梯度更新参数。
        optimizer.step()

        # 计算当前批次的精度。   第一维度是0，第二维度是1
        acc = (logits.argmax(dim=-1) == labels).float().mean()
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
        # 记录损失和准确度
        train_loss.append(loss.item())
        train_accs.append(acc)

    # 训练集的平均损失和精度是记录值的平均值。
    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)
    # 更新学习率
    scheduler.step(train_loss)
    print("第%d个epoch的学习率：%f" % (epoch + 1, optimizer.param_groups[0]['lr']))

    # 打印信息
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")
# ---------- test ----------
    model.eval()
    test_loss = []
    test_accs = []
    for batch in tqdm(test_Dataloader):
        imgs, labels = batch

        # 不需要梯度验证.
        # Using  torch.no_grad()  accelerates the forward process.
        with torch.no_grad():
            logits = model(imgs.to(device))

        # 我们仍然可以计算损失（但不能计算梯度）。
        loss = criterion(logits, labels.to(device))

        # 计算当前批次的精度。
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # 记录损失和准确性
        test_loss.append(loss.item())
        test_accs.append(acc)

    # 整个验证集的平均损失和准确度是记录值的平均值
    test_loss = sum(test_loss) / len(test_loss)
    test_accs = sum(test_accs) / len(test_accs)

    # 打印信息.
    print(f"[ test ]  loss = {test_loss:.5f}, acc = {test_accs:.5f}")
  # 如果模型改进了，在这个时间点保存一个检查点
    if test_accs > best_acc:
        best_acc = test_accs
        net_name="_resnet10_"
        path=model_path+"model_weights_2d_"+f"{net_name}"+f'{best_acc}'+".pth"
        torch.save(model.state_dict(), path)
        print('saving model with acc {:.3f}'.format(best_acc))


torch.Size([21564, 3, 32, 32]) torch.Size([21564])
torch.Size([3008, 3, 32, 32]) torch.Size([3008])
Finished reading the train set of Leaves Dataset (21564 samples found)
Finished reading the valid set of Leaves Dataset (0 samples found)
Finished reading the test set of Leaves Dataset (3008 samples found)
cpu
<bound method Module.parameters of ResNet(
  (conv1): Sequential(
    (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(24, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxp

100%|██████████| 1347/1347 [02:46<00:00,  8.10it/s]


第1个epoch的学习率：0.000099
[ Train | 001/050 ] loss = 0.33360, acc = 0.88479


100%|██████████| 188/188 [00:02<00:00, 64.11it/s]


[ test ]  loss = 0.38181, acc = 0.83544
saving model with acc 0.835


...

[ test ]  loss = 2.06628, acc = 0.85439


100%|██████████| 1347/1347 [02:57<00:00,  7.61it/s]


第50个epoch的学习率：0.000100
[ Train | 050/050 ] loss = 0.17969, acc = 0.94219


100%|██████████| 188/188 [00:02<00:00, 64.66it/s]


[ test ]  loss = 2.08677, acc = 0.84641


In [104]:
import h5py
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import timm  # timm库有更丰富的预训练模型
from torch import nn, optim
from tqdm import tqdm
import random
# 设置文件路径并得到数据集
train_data_file = "../CNN/data/train_data/"
test_data_file = "../CNN/data/test_data/"
# model_path   torch.save(model.state_dict(), '../CNN/Cmodel_weights.pth')
model_path = "../CNN/"

# 超参数
learning_rate = 1e-3  # 1e-4
weight_decay = 1e-4
batch_size = 16
n_epochs = 50
best_acc= 0
#提取数据
def data_image_label(mode):  #shuffle
        if mode == 'train':
            for i in range(1, 9):
                date_path = f"{train_data_file}" + "traindata" + f"{i}" + ".mat"
                if i != 1:
                    data_image = np.append(data_image, np.array(h5py.File(date_path)["video"], dtype=np.float32),axis=0)
                    data_label = np.append(data_label, np.array(h5py.File(date_path)["label"], dtype=np.float32),axis=0)
                else:
                    data_image = np.array(h5py.File(date_path)["video"], dtype=np.float32)
                    data_label = np.array(h5py.File(date_path)["label"], dtype=np.float32)
            #shuffle
            index = [i for i in range(len(data_image))]
            random.shuffle(index)
            random.shuffle(index)
            data_image = data_image[index,:,:,:]
            data_label = data_label[index,:]


            image_tensor = torch.tensor(data_image, dtype=torch.float32)
            label_tensor = torch.tensor(data_label.squeeze(), dtype=torch.long)
            real_len = len(image_tensor)
            print(image_tensor.shape, label_tensor.shape)
            #  #valid_date
            # valid_image_tensor=image_tensor[:self.valid_len,:,:,:]
            # valid_label_tensor=label_tensor[:self.valid_len]
            # #train_date
            # image_tensor=self.image_tensor[self.valid_len:,:,:,:]
            # label_tensor=self.label_tensor[self.valid_len:]
            # real_len=len(self.image_tensor)
            return image_tensor,label_tensor,real_len
        elif mode == "test":
            for i in range(1, 5):
                date_path = f"{test_data_file}" + "testdata" + f"{i}" + "_label.mat"
                if i != 1:
                    data_image = np.append(data_image, np.array(h5py.File(date_path)["video"], dtype=np.float32),axis=0)
                    data_label = np.append(data_label, np.array(h5py.File(date_path)["label"], dtype=np.float32),axis=0)
                else:
                    data_image = np.array(h5py.File(date_path)["video"], dtype=np.float32)
                    data_label = np.array(h5py.File(date_path)["label"], dtype=np.float32)
            image_tensor = torch.tensor(data_image, dtype=torch.float32)
            label_tensor = torch.tensor(data_label.squeeze(), dtype=torch.long)
            real_len = len(image_tensor)
            print(image_tensor.shape, label_tensor.shape)
            return image_tensor,label_tensor,real_len
train_image_tensor,train_label_tensor,train_real_len=data_image_label('train')
test_image_tensor,test_label_tensor,test_real_len=data_image_label('test')

class SAPI_FMF_Dataset(Dataset):
    def __init__(self,image_data,label_data,data_len,mode="train", valid_ratio=0):
        """
        Args:
            file_path (string): 图像文件 路径
            mode (string): 训练模式还是测试模式
            valid_ratio (float): 验证集比例
        """
        self.image_tensor=image_data
        self.label_tensor=label_data
        self.real_len=data_len
        self.mode = mode
        self.valid_ratio = valid_ratio
        self.valid_len=int(self.valid_ratio*self.real_len)
        if self.mode == 'train':
            self.image_tensor=self.image_tensor[self.valid_len:,:,:,:]
            self.label_tensor=self.label_tensor[self.valid_len:]
            self.real_len=len(self.image_tensor)
        elif self.mode == 'valid':
            self.valid_image_tensor=self.image_tensor[:self.valid_len,:,:,:]
            self.valid_label_tensor=self.label_tensor[:self.valid_len]
            self.real_len=len(self.valid_image_tensor)
        elif self.mode == 'test':
            pass
        print('Finished reading the {} set of Leaves Dataset ({} samples found)'.format(mode, self.real_len))
    def __getitem__(self, index):
        # 从 image_arr中得到索引对应的文件名
        return self.image_tensor[index, :, :, :], self.label_tensor[index]
    def __len__(self):
        return self.real_len  # self.real_len = len(self.image_arr) 返回的是训练/验证/测试/图像的数量
train_dataset = SAPI_FMF_Dataset(train_image_tensor,train_label_tensor,train_real_len,mode='train')
valid_dataset = SAPI_FMF_Dataset( train_image_tensor,train_label_tensor,train_real_len,mode='valid')
test_dataset = SAPI_FMF_Dataset( test_image_tensor,test_label_tensor,test_real_len,mode='test')


# 定义data loader
train_Dataloader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True,drop_last=True)  # 打开乱序  Falsenum_workers=0)
valid_Dataloader = torch.utils.data.DataLoader(dataset=valid_dataset,batch_size=batch_size,shuffle=False,drop_last=True)   #打开乱序  Falsenum_workers=0)
test_Dataloader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False,drop_last=False)
# GPU计算
def get_device():
    return 'cuda' if torch.cuda.is_available() else 'cpu'
device = get_device()
print(device)

model = timm.create_model("skresnet18",num_classes=2,in_chans=3,pretrained=True,output_stride=8,drop_rate=.5,drop_path_rate=0.5)
# model = timm.create_model("resnet18")
model.fc = nn.Sequential(
    nn.Dropout(.5),
    nn.Linear(model.fc.in_features, 2),
    nn.Dropout(.5)
)
model.to(device)
model.device = device
optimizer = optim.AdamW(model.parameters(), lr=1e-4,
                        weight_decay=weight_decay)  # lr=learning_rate,weight_decay= weight_decay
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=2)
print(model.parameters)
# train
# 对于分类任务，我们使用交叉熵作为性能的度量。
criterion = nn.CrossEntropyLoss()


for epoch in range(n_epochs):
    # 记录训练信息
    model.train()
    train_loss = []
    train_accs = []
    for batch in tqdm(train_Dataloader):
        # ---------- Training ----------  以下是训练模型
        imgs, labels = batch
        # print(imgs.shape)

        logits = model(imgs)  # 将图形数据带入模型计算预测
        # 计算交叉熵损失
        loss = criterion(logits, labels)
        # 清除上一步中存储在参数中的梯度。
        optimizer.zero_grad()
        # 计算参数的梯度。
        loss.backward()
        # 用计算的梯度更新参数。
        optimizer.step()

        # 计算当前批次的精度。   第一维度是0，第二维度是1
        acc = (logits.argmax(dim=-1) == labels).float().mean()
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
        # 记录损失和准确度
        train_loss.append(loss.item())
        train_accs.append(acc)

    # 训练集的平均损失和精度是记录值的平均值。
    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)
    # 更新学习率
    scheduler.step(train_loss)
    print("第%d个epoch的学习率：%f" % (epoch + 1, optimizer.param_groups[0]['lr']))

    # 打印信息
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")
# ---------- test ----------
    model.eval()
    test_loss = []
    test_accs = []
    for batch in tqdm(test_Dataloader):
        imgs, labels = batch

        # 不需要梯度验证.
        # Using  torch.no_grad()  accelerates the forward process.
        with torch.no_grad():
            logits = model(imgs.to(device))

        # 我们仍然可以计算损失（但不能计算梯度）。
        loss = criterion(logits, labels.to(device))

        # 计算当前批次的精度。
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # 记录损失和准确性
        test_loss.append(loss.item())
        test_accs.append(acc)

    # 整个验证集的平均损失和准确度是记录值的平均值
    test_loss = sum(test_loss) / len(test_loss)
    test_accs = sum(test_accs) / len(test_accs)

    # 打印信息.
    print(f"[ test ]  loss = {test_loss:.5f}, acc = {test_accs:.5f}")
  # 如果模型改进了，在这个时间点保存一个检查点
    if test_accs > best_acc:
        best_acc = test_accs
        net_name="skresnet18"
        path=model_path+"model_weights_2d_"+f"{net_name}"+f'{best_acc}'+".pth"
        torch.save(model.state_dict(), path)
        print('saving model with acc {:.3f}'.format(best_acc))


torch.Size([21564, 3, 32, 32]) torch.Size([21564])
torch.Size([3008, 3, 32, 32]) torch.Size([3008])
Finished reading the train set of Leaves Dataset (21564 samples found)
Finished reading the valid set of Leaves Dataset (0 samples found)
Finished reading the test set of Leaves Dataset (3008 samples found)
cpu
<bound method Module.parameters of ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): SelectiveKernelBasic(
      (conv1): SelectiveKernel(
        (paths): ModuleList(
          (0): ConvNormActAa(
            (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn): BatchNormAct2d(
              64, eps=1e-05, momentum=0.1, affine=True, track_running_stat

100%|██████████| 1347/1347 [06:41<00:00,  3.35it/s]


第1个epoch的学习率：0.000099
[ Train | 001/050 ] loss = 0.36953, acc = 0.87036


100%|██████████| 188/188 [00:06<00:00, 28.35it/s]


[ test ]  loss = 0.81996, acc = 0.83078
saving model with acc 0.831


...

[ test ]  loss = 1.82385, acc = 0.69781


100%|██████████| 1347/1347 [06:31<00:00,  3.44it/s]


第50个epoch的学习率：0.000100
[ Train | 050/050 ] loss = 0.18259, acc = 0.93898


100%|██████████| 188/188 [00:06<00:00, 27.81it/s]


[ test ]  loss = 2.28513, acc = 0.69581


In [105]:
import h5py
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import timm  # timm库有更丰富的预训练模型
from torch import nn, optim
from tqdm import tqdm
import random
# 设置文件路径并得到数据集
train_data_file = "../CNN/data/train_data/"
test_data_file = "../CNN/data/test_data/"
# model_path   torch.save(model.state_dict(), '../CNN/Cmodel_weights.pth')
model_path = "../CNN/"

# 超参数
learning_rate = 1e-3  # 1e-4
weight_decay = 1e-4
batch_size = 16
n_epochs = 50
best_acc= 0
#提取数据
def data_image_label(mode):  #shuffle
        if mode == 'train':
            for i in range(1, 9):
                date_path = f"{train_data_file}" + "traindata" + f"{i}" + ".mat"
                if i != 1:
                    data_image = np.append(data_image, np.array(h5py.File(date_path)["video"], dtype=np.float32),axis=0)
                    data_label = np.append(data_label, np.array(h5py.File(date_path)["label"], dtype=np.float32),axis=0)
                else:
                    data_image = np.array(h5py.File(date_path)["video"], dtype=np.float32)
                    data_label = np.array(h5py.File(date_path)["label"], dtype=np.float32)
            #shuffle
            index = [i for i in range(len(data_image))]
            random.shuffle(index)
            random.shuffle(index)
            data_image = data_image[index,:,:,:]
            data_label = data_label[index,:]


            image_tensor = torch.tensor(data_image, dtype=torch.float32)
            label_tensor = torch.tensor(data_label.squeeze(), dtype=torch.long)
            real_len = len(image_tensor)
            print(image_tensor.shape, label_tensor.shape)
            #  #valid_date
            # valid_image_tensor=image_tensor[:self.valid_len,:,:,:]
            # valid_label_tensor=label_tensor[:self.valid_len]
            # #train_date
            # image_tensor=self.image_tensor[self.valid_len:,:,:,:]
            # label_tensor=self.label_tensor[self.valid_len:]
            # real_len=len(self.image_tensor)
            return image_tensor,label_tensor,real_len
        elif mode == "test":
            for i in range(1, 5):
                date_path = f"{test_data_file}" + "testdata" + f"{i}" + "_label.mat"
                if i != 1:
                    data_image = np.append(data_image, np.array(h5py.File(date_path)["video"], dtype=np.float32),axis=0)
                    data_label = np.append(data_label, np.array(h5py.File(date_path)["label"], dtype=np.float32),axis=0)
                else:
                    data_image = np.array(h5py.File(date_path)["video"], dtype=np.float32)
                    data_label = np.array(h5py.File(date_path)["label"], dtype=np.float32)
            image_tensor = torch.tensor(data_image, dtype=torch.float32)
            label_tensor = torch.tensor(data_label.squeeze(), dtype=torch.long)
            real_len = len(image_tensor)
            print(image_tensor.shape, label_tensor.shape)
            return image_tensor,label_tensor,real_len
train_image_tensor,train_label_tensor,train_real_len=data_image_label('train')
test_image_tensor,test_label_tensor,test_real_len=data_image_label('test')

class SAPI_FMF_Dataset(Dataset):
    def __init__(self,image_data,label_data,data_len,mode="train", valid_ratio=0):
        """
        Args:
            file_path (string): 图像文件 路径
            mode (string): 训练模式还是测试模式
            valid_ratio (float): 验证集比例
        """
        self.image_tensor=image_data
        self.label_tensor=label_data
        self.real_len=data_len
        self.mode = mode
        self.valid_ratio = valid_ratio
        self.valid_len=int(self.valid_ratio*self.real_len)
        if self.mode == 'train':
            self.image_tensor=self.image_tensor[self.valid_len:,:,:,:]
            self.label_tensor=self.label_tensor[self.valid_len:]
            self.real_len=len(self.image_tensor)
        elif self.mode == 'valid':
            self.valid_image_tensor=self.image_tensor[:self.valid_len,:,:,:]
            self.valid_label_tensor=self.label_tensor[:self.valid_len]
            self.real_len=len(self.valid_image_tensor)
        elif self.mode == 'test':
            pass
        print('Finished reading the {} set of Leaves Dataset ({} samples found)'.format(mode, self.real_len))
    def __getitem__(self, index):
        # 从 image_arr中得到索引对应的文件名
        return self.image_tensor[index, :, :, :], self.label_tensor[index]
    def __len__(self):
        return self.real_len  # self.real_len = len(self.image_arr) 返回的是训练/验证/测试/图像的数量
train_dataset = SAPI_FMF_Dataset(train_image_tensor,train_label_tensor,train_real_len,mode='train')
valid_dataset = SAPI_FMF_Dataset( train_image_tensor,train_label_tensor,train_real_len,mode='valid')
test_dataset = SAPI_FMF_Dataset( test_image_tensor,test_label_tensor,test_real_len,mode='test')


# 定义data loader
train_Dataloader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True,drop_last=True)  # 打开乱序  Falsenum_workers=0)
valid_Dataloader = torch.utils.data.DataLoader(dataset=valid_dataset,batch_size=batch_size,shuffle=False,drop_last=True)   #打开乱序  Falsenum_workers=0)
test_Dataloader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False,drop_last=False)
# GPU计算
def get_device():
    return 'cuda' if torch.cuda.is_available() else 'cpu'
device = get_device()
print(device)

model = timm.create_model("legacy_seresnet18",num_classes=2,in_chans=3,pretrained=True,drop_rate=.5)
# model = timm.create_model("resnet18")
# model.fc = nn.Sequential(
#     nn.Dropout(.5),
#     nn.Linear(model.fc.in_features, 2),
#     nn.Dropout(.5)
# )
model.to(device)
model.device = device
optimizer = optim.AdamW(model.parameters(), lr=1e-4,
                        weight_decay=weight_decay)  # lr=learning_rate,weight_decay= weight_decay
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=2)
print(model.parameters)
# train
# 对于分类任务，我们使用交叉熵作为性能的度量。
criterion = nn.CrossEntropyLoss()


for epoch in range(n_epochs):
    # 记录训练信息
    model.train()
    train_loss = []
    train_accs = []
    for batch in tqdm(train_Dataloader):
        # ---------- Training ----------  以下是训练模型
        imgs, labels = batch
        # print(imgs.shape)

        logits = model(imgs)  # 将图形数据带入模型计算预测
        # 计算交叉熵损失
        loss = criterion(logits, labels)
        # 清除上一步中存储在参数中的梯度。
        optimizer.zero_grad()
        # 计算参数的梯度。
        loss.backward()
        # 用计算的梯度更新参数。
        optimizer.step()

        # 计算当前批次的精度。   第一维度是0，第二维度是1
        acc = (logits.argmax(dim=-1) == labels).float().mean()
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
        # 记录损失和准确度
        train_loss.append(loss.item())
        train_accs.append(acc)

    # 训练集的平均损失和精度是记录值的平均值。
    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)
    # 更新学习率
    scheduler.step(train_loss)
    print("第%d个epoch的学习率：%f" % (epoch + 1, optimizer.param_groups[0]['lr']))

    # 打印信息
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")
# ---------- test ----------
    model.eval()
    test_loss = []
    test_accs = []
    for batch in tqdm(test_Dataloader):
        imgs, labels = batch

        # 不需要梯度验证.
        # Using  torch.no_grad()  accelerates the forward process.
        with torch.no_grad():
            logits = model(imgs.to(device))

        # 我们仍然可以计算损失（但不能计算梯度）。
        loss = criterion(logits, labels.to(device))

        # 计算当前批次的精度。
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # 记录损失和准确性
        test_loss.append(loss.item())
        test_accs.append(acc)

    # 整个验证集的平均损失和准确度是记录值的平均值
    test_loss = sum(test_loss) / len(test_loss)
    test_accs = sum(test_accs) / len(test_accs)

    # 打印信息.
    print(f"[ test ]  loss = {test_loss:.5f}, acc = {test_accs:.5f}")
  # 如果模型改进了，在这个时间点保存一个检查点
    if test_accs > best_acc:
        best_acc = test_accs
        net_name="legacy_seresnet18"
        path=model_path+"model_weights_2d_"+f"{net_name}"+f'{best_acc}'+".pth"
        torch.save(model.state_dict(), path)
        print('saving model with acc {:.3f}'.format(best_acc))


torch.Size([21564, 3, 32, 32]) torch.Size([21564])
torch.Size([3008, 3, 32, 32]) torch.Size([3008])
Finished reading the train set of Leaves Dataset (21564 samples found)
Finished reading the valid set of Leaves Dataset (0 samples found)
Finished reading the test set of Leaves Dataset (3008 samples found)
cpu
<bound method Module.parameters of SENet(
  (layer0): Sequential(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
  )
  (pool0): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (layer1): Sequential(
    (0): SEResNetBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=F

100%|██████████| 1347/1347 [05:42<00:00,  3.93it/s]


第1个epoch的学习率：0.000100
[ Train | 001/050 ] loss = 0.08521, acc = 0.96817


100%|██████████| 188/188 [00:04<00:00, 41.50it/s]


[ test ]  loss = 1.43355, acc = 0.81516
saving model with acc 0.815


...

[ test ]  loss = 2.12883, acc = 0.78923


100%|██████████| 1347/1347 [05:54<00:00,  3.80it/s]


第50个epoch的学习率：0.000100
[ Train | 050/050 ] loss = 0.00142, acc = 0.99958


100%|██████████| 188/188 [00:04<00:00, 40.76it/s]

[ test ]  loss = 2.75506, acc = 0.72972


In [107]:
def test_data(i):
    date_path = f"{test_data_file}" + "testdata" + f"{i}" + ".mat"
    data_image = np.array(h5py.File(date_path)["video"], dtype=np.float32)
    image_tensor = torch.tensor(data_image, dtype=torch.float32)
    real_len = len(image_tensor)
    print(image_tensor.shape,real_len)
    return image_tensor,real_len

class test_result_Dataset(Dataset):
    def __init__(self,image_data,data_len):
        """
        Args:
            file_path (string): 图像文件 路径
        """
        self.image_tensor=image_data
        self.real_len=data_len
        print('Finished reading the  Dataset ({} samples found)'.format(self.real_len))
    def __getitem__(self, index):
        # 从 image_arr中得到索引对应的文件名
        return self.image_tensor[index, :, :, :]
    def __len__(self):
        return self.real_len

model = timm.create_model("resnet10t",num_classes=2,in_chans=3,pretrained=True,output_stride=8,drop_rate=.5,drop_path_rate=0.5,drop_block_rate=0)
# model = timm.create_model("resnet18")
model.fc = nn.Sequential(
    nn.Dropout(.5),
    nn.Linear(model.fc.in_features, 2),
    nn.Dropout(.5)
)
model_path="../CNN/model_weights_2d__resnet10_0.8354388475418091.pth"
model.load_state_dict(torch.load(model_path))
model.eval()
for i in range(5,9):
    test_data5_image,test_data5_len=test_data(i)
    test_dataset5 = test_result_Dataset(test_data5_image,test_data5_len)
    test_Dataloader5 = torch.utils.data.DataLoader(dataset=test_dataset5, batch_size=batch_size, shuffle=False,drop_last=False)
    predictions = []
    for batch in tqdm(test_Dataloader5):
        imgs = batch
        with torch.no_grad():
            logits = model(imgs.to(device))
        predictions.extend(logits.argmax(dim=-1).cpu().numpy().tolist())
    print(predictions)



torch.Size([601, 3, 32, 32]) 601
Finished reading the  Dataset (601 samples found)


100%|██████████| 38/38 [00:00<00:00, 71.60it/s]


[0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

100%|██████████| 63/63 [00:00<00:00, 63.80it/s]


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

100%|██████████| 33/33 [00:00<00:00, 63.68it/s]


[1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

100%|██████████| 39/39 [00:00<00:00, 68.84it/s]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [112]:
import pandas as pd
def test_data(i):
    date_path = f"{test_data_file}" + "testdata" + f"{i}" + ".mat"
    data_image = np.array(h5py.File(date_path)["video"], dtype=np.float32)
    image_tensor = torch.tensor(data_image, dtype=torch.float32)
    real_len = len(image_tensor)
    print(image_tensor.shape,real_len)
    return image_tensor,real_len

class test_result_Dataset(Dataset):
    def __init__(self,image_data,data_len):
        """
        Args:
            file_path (string): 图像文件 路径
        """
        self.image_tensor=image_data
        self.real_len=data_len
        print('Finished reading the  Dataset ({} samples found)'.format(self.real_len))
    def __getitem__(self, index):
        # 从 image_arr中得到索引对应的文件名
        return self.image_tensor[index, :, :, :]
    def __len__(self):
        return self.real_len

model = timm.create_model("resnet10t",num_classes=2,in_chans=3,pretrained=True,output_stride=8,drop_rate=.5,drop_path_rate=0.5,drop_block_rate=0)
# model = timm.create_model("resnet18")
model.fc = nn.Sequential(
    nn.Dropout(.5),
    nn.Linear(model.fc.in_features, 2),
    nn.Dropout(.5)
)
model_path="../CNN/model_weights_2d__resnet10_0.8447473645210266.pth"
model.load_state_dict(torch.load(model_path))
model.eval()
for i in range(5,9):
    test_data_image,test_data_len=test_data(i)
    test_dataset = test_result_Dataset(test_data_image,test_data_len)
    test_Dataloader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False,drop_last=False)
    predictions = []
    for batch in tqdm(test_Dataloader):
        imgs = batch
        with torch.no_grad():
            logits = model(imgs.to(device))
        predictions.extend(logits.argmax(dim=-1).cpu().numpy().tolist())
    excel_path="../CNN/data/test_data/testdata"+f"{i}"+"_cls.xlsx"
    index=np.array(range(1,test_data_len+1)).tolist()
    data_toexcel={"index":index,"result":predictions}
    data_toexcel = pd.DataFrame(data_toexcel)
    data_toexcel.to_excel(excel_path, sheet_name="sheet1",index=False)
    print(predictions)

torch.Size([601, 3, 32, 32]) 601
Finished reading the  Dataset (601 samples found)


100%|██████████| 38/38 [00:00<00:00, 65.40it/s]


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

100%|██████████| 63/63 [00:00<00:00, 74.94it/s]


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

100%|██████████| 33/33 [00:00<00:00, 70.32it/s]


[0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

100%|██████████| 39/39 [00:00<00:00, 62.81it/s]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 